In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [57]:
class Spatial_Block(nn.Module):
    '''
    inputsize: Batchsize x Dimension x Length
    
    input_dimension: 
    pool_kernel_size: pooling layer size 
    d_prime: fc convert size
    
    outputsize: Batchsize x Dimension x Length
    '''
    def __init__(self, input_dimension, pool_kernel_size, d_prime):
        super().__init__()
        self.pooling = nn.AvgPool1d(kernel_size=pool_kernel_size, stride=1, padding=(pool_kernel_size-1)//2)
        self.fc1 = nn.Linear(in_features=input_dimension, out_features=d_prime)
        self.fc2 = nn.Linear(in_features=d_prime, out_features=input_dimension)
    
    def forward(self, x):
        y = self.pooling(x)
        y = y.transpose(1,2)
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        y = y.transpose(1,2)
        return x * y
        
class SMATE_Encoder(nn.Module):
    '''
    input_shape: batch_size x Dimension x Length
    output_shape:
    
    input_dimension: int, the channels/dimension of the input 
    input_length: int, the length of the input 
    num_layers: int, the num of blocks 
    d_prime: List, the hidden size of Spatial block hidden layer, the length of this list should be same as the num_layers
    kernels: List, the kernel sizes of each SMATE_Encoder block(both the Convd1d/AvgPooling), the length of this list should be same as the num_layers
    out_channels: List, the channels/dimensions of the output of each block,  the length of this list should be same as the num_layers
    rnn: Str, optional, default as GRU, the type of RNN in SMATE_Encoder, one of GRU, LSTM
    
    '''
    def __init__(self, input_dimension, input_length, d_prime, kernels, out_channels, final_pool, num_layers=3, rnn="GRU"):
        super().__init__()
        if rnn is "GRU":
            self.rnn_sq = nn.GRU(input_size=input_dimension, batch_first=True, hidden_size=out_channels[-1], num_layers=num_layers)
        elif rnn is "LSTM":
            self.rnn_sq = nn.LSTM(input_size=input_dimension, batch_first=True, hidden_size=out_channels[-1], num_layers=num_layers)
        out_channels.insert(0, input_dimension)
        self.smate = nn.Sequential()
        for i in range(num_layers):
            self.smate.add_module("smate_block"+str(i), Spatial_Block(out_channels[i], pool_kernel_size=kernels[i], d_prime=d_prime))
            self.smate.add_module("conv1d"+str(i), nn.Conv1d(in_channels=out_channels[i], out_channels=out_channels[i+1], kernel_size=kernels[i], padding=(kernels[i]-1)//2))
            self.smate.add_module("batch_norm"+str(i), nn.BatchNorm1d(out_channels[i+1]))
            self.smate.add_module("relu"+str(i), nn.ReLU())
        self.t_avgpool = nn.AvgPool1d(kernel_size=final_pool, padding=(final_pool-1)//2)
        self.s_avgpool = nn.AvgPool1d(kernel_size=final_pool, padding=(final_pool-1)//2)
        self.mlp = nn.Sequential()
        self.mlp.add_module("fc1", nn.Linear(2*out_channels[-1], out_features=500))
        self.mlp.add_module("relu", nn.ReLU())
        self.mlp.add_module("fc2", nn.Linear(500, 300))
        self.mlp.add_module("relu", nn.ReLU())
    def forward(self, x):
        x_transpose = x.transpose(1,2)
        y_rnn,_ = self.rnn_sq(x_transpose)
        y_s = self.smate(x)
        y_s = y_s.transpose(1,2)
        y = torch.cat([y_s, y_rnn], dim=2)
        y = self.mlp(y)
        return y
    
class SMATE_Decoder(nn.Module):
    def __init__(self, num_layers, input_dimension, input_length, hidden_size, rnn="GRU"):
        '''
        input_shape: batch_size X Length X Dimension
        ouput_shape: batch_size x Dimension x Length
        
        input_dimension: int, the channels/dimension of the input 
        input_length: int, the length of the input 
        num_layers: int, the num of blocks 
        d_prime: List, the hidden size of Spatial block hidden layer, the length of this list should be same as the num_layers
        kernels: List, the kernel sizes of each SMATE_Encoder block(both the Convd1d/AvgPooling), the length of this list should be same as the num_layers
        out_channels: List, the channels/dimensions of the output of each block,  the length of this list should be same as the num_layers
        rnn: Str, optional, default as GRU, the type of RNN in SMATE_Encoder, one of GRU, LSTM
        '''
        super().__init__()
        # self.uppool = nn.Upsample()
        if rnn is "GRU":
            self.rnn_sq = nn.GRU(input_size=input_dimension, batch_first=True, hidden_size=hidden_size, num_layers=num_layers)
        elif rnn is "LSTM":
            self.rnn_sq = nn.LSTM(input_size=input_dimension, batch_first=True, hidden_size=hidden_size, num_layers=num_layers) 
    
    def forward(self, x):
        # y = self.uppool(x)
        y,(h,c) = self.rnn_sq(x)
        return y
def euclidean_dist(x, y):
    # x: N x D
    # y: M x D
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    assert d == y.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)
    
class ClusterML(nn.Module):
    def __init__(self, nclass, input_size, embedding_size):
        super().__init__()
        self.centroid_List = nn.Parameter(torch.zeros([nclass, embedding_size]))
        self.flag = [0 for i in range(nclass)]
        self.nclass = nclass
        self.linear = nn.Linear(input_size ,embedding_size)
    
    def forward(self, x, y):
        output = self.linear(x)
        for i in range(self.nclass):
            idx = (y == i)
            batch_repr = output[idx].mean(0)
            if self.flag[i]:
                self.centroid_List[i] += -0.1 * self.centroid_List[i] + 0.1 * batch_repr
            else:
                self.centroid_List[i] += batch_repr
            proto_dist = euclidean_dist(self.centroid_List, self.centroid_List)
            proto_dist = torch.exp(-0.5 * proto_dist)
            dist = euclidean_dist(output, self.centroid_List)
        return F.sigmoid(torch.exp(-0.5 * dist)), proto_dist

def output_conv2d_size(in_size, kernel_size, stride=1, padding=0, dilation=1):
    output = []
    for i in range(2):
        output.append((in_size[i]-kernel_size[i]-(dilation-1)*(kernel_size[i]-1)+2*padding)//stride +1)
    return output


class TDConvdBlock(nn.Module):
    def __init__(self, inchannels, outchannels, input_shape, stride, kernel_size=[2,2], use_FAM=True):
        super().__init__()
        self.out_channels = outchannels
        self.outputsize = output_conv2d_size(input_shape, kernel_size,stride=stride)
        self.convd = nn.Conv2d(in_channels=inchannels, out_channels=outchannels, stride=stride, kernel_size=kernel_size)
        
        self.feature_att_fmap = nn.Linear(input_shape[0]*input_shape[1], self.outputsize[0]*self.outputsize[1])
        self.feature_att_kernel = nn.Linear(kernel_size[0]*kernel_size[1]*inchannels, self.outputsize[0]*self.outputsize[1])
        
    
    def forward(self, x):
        '''
        x size: batchsize x inchannel x length x width 
        '''
        y = self.convd(x)
        kernel_graphs = []
        featuremap_graphs = []
        N = x.size(0)
        
        for i in range(self.out_channels):
            #kernel_graph size: inchannels x kernel_length x kernel_width 
            kernel_graph = self.convd.state_dict()['weight'][i]
            kernel_graphs.append(torch.unsqueeze(torch.flatten(kernel_graph),dim=0))
            #featuremap_graph size: batchsize x inchannels x in_featuremap_length x in_featuremap_width 
            featuremap_graph = x[:,1,:]
            featuremap_graphs.append(torch.unsqueeze(featuremap_graph.view(N, -1),dim=1))
        # kernel_graphs cat size: out_channels x kernelshape(eq. length * width)
        # attn_kernel size: batchsize x out_channels x kernelshape(eq. length * width)
        attn_kernel =  (torch.unsqueeze(torch.cat(kernel_graphs, dim=0),dim=0)).expand(N,-1,-1)
        
        # featuremap_graphs cat size: batch_size x outchannels x featuremapshape(eq. length * width)
        attn_featuremap = torch.cat(featuremap_graphs, dim=1)
        # attn_score size:   
        
        a1 = self.feature_att_fmap(attn_featuremap)
        a2 = self.feature_att_kernel(attn_kernel) 
        print(a1.shape, a2.shape) 
        attn_score = torch.tanh(a1+a2)
        attn_weight = F.softmax(attn_score, dim=2)
        attn_weight = torch.reshape(attn_weight, [self.out_channels, self.outputsize[0], -1])
        y = y*attn_weight
        return y

class LocalAttentionModule(nn.Module):
    def __init__(self, ecg_seg_length, height, cw):
        super().__init__()
        self.ecg_seg_linear = nn.Linear(ecg_seg_length, height)
        self.featuremap = nn.Linear(cw, 1)
        self.height = height
    def forward(self, x, ecg_seg):
        '''
        input_size : batch_size x channels x width x height
        ecg_seg : batch_size x ecg_length
        '''
        N = x.size(0)
        # y size: batch_size x height x (width*channels)
        y = torch.transpose(x, 1, 3)
        y = torch.reshape(y, [N, self.height, -1])

        out = torch.squeeze(self.featuremap(y), -1)
        ecg_out = self.ecg_seg_linear(ecg_seg)

        attn = torch.tanh(out+ecg_out)
        attn_score = F.softmax(attn, dim=1)
        attn_score = torch.unsqueeze(torch.unsqueeze(attn_score, 1), 1)
        return x * attn_score

class TestConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv1d(3, 10, 3)
        self.linear = nn.Linear(9,1)
        self.linear2 = nn.Linear(180,2)
    
    def forward(self,x):
        N = x.size(0)
        y = self.conv(x)
        graphs = []
        for i in range(10):
            # weight size: out_channels x in_channels x kernel_size
            # kernel_graph size: in_channels x kernel_size
            kernel_graph = self.conv.state_dict()['weight'][i]
            # graph size: total_size
            graph = torch.flatten(kernel_graph)
            graphs.append(torch.unsqueeze(graph, dim=0))
            # torch.cat([graphs, ])
         
        graphs = torch.cat(graphs, dim=0)    
        att = self.linear(graphs)
        att_weihght = F.softmax(att,dim=1)
        att_weihght = att_weihght.expand_as(y)
        z= (y * att_weihght).view(N, -1)

        return  self.linear2(z)  

def three(num_inputs, num_hiddens):
    return (
        nn.Parameter(torch.randn(num_inputs, num_hiddens), requires_grad=True),
        nn.Parameter(torch.randn(num_hiddens, num_hiddens), requires_grad=True),
        nn.Parameter(torch.zeros(num_hiddens), requires_grad=True)
    )
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, bidirectional=False):
        super().__init__()
        self.W_xi, self.W_hi, self.b_i = three(input_size, hidden_size)  # 输入门参数
        self.W_xf, self.W_hf, self.b_f = three(input_size, hidden_size)  # 遗忘门参数
        self.W_xo, self.W_ho, self.b_o = three(input_size, hidden_size)  # 输出门参数
        self.W_xc, self.W_hc, self.b_c = three(input_size, hidden_size)  # 候选记忆单元参数
        self.W_hq = nn.Parameter(torch.randn(hidden_size, input_size), requires_grad=True)
        self.b_q = nn.Parameter(torch.zeros(input_size), requires_grad=True)
        self.hidden_size = hidden_size
    def forward(self, inputs):
        N = inputs.size(1)
        (H, C) = (torch.zeros(N, self.hidden_size), torch.zeros(N, self.hidden_size))
        outputs = []
        for X in inputs:
            I = torch.sigmoid((X @ self.W_xi) + (H @ self.W_hi) + self.b_i)
            F = torch.sigmoid((X @ self.W_xf) + (H @ self.W_hf) + self.b_f)
            O = torch.sigmoid((X @ self.W_xo) + (H @ self.W_ho) + self.b_o)
            C_tilda = torch.tanh((X @ self.W_xc) + (H @ self.W_hc) + self.b_c)
            C = F * C + I * C_tilda
            H = O * torch.tanh(C)
            Y = (H @ self.W_hq) + self.b_q
            outputs.append(torch.unsqueeze(Y, dim=0))
            
        return torch.cat(outputs, dim=0), (H, C)


In [58]:
a = torch.Tensor(2,3,5)
l = nn.Linear(5, 1)

attn_score = torch.randn(4,5)
attn_score = torch.unsqueeze(torch.unsqueeze(attn_score, 1), 1)
lam = LocalAttentionModule(30,6,40)
ecg_seg = torch.randn(3,30)
featuremap = torch.randn(3, 8, 5, 6)
g = lam(featuremap, ecg_seg)
print(g.size())
attn_score.size()
lstm = LSTM(5, 6)
k,(_,_) = lstm(a) 
k.size()

torch.Size([3, 8, 5, 6])


torch.Size([2, 3, 5])

In [26]:
tdb = TDConvdBlock(3, 6, [20,50],2,[3,3])
x = torch.randn(10,3,20,50)
tdb(x).shape
optimizer_ExpLR = torch.optim.SGD()
ExpLR = torch.optim.lr_scheduler.ExponentialLR(optimizer_ExpLR, gamma=0.98)

torch.Size([10, 6, 216]) torch.Size([10, 6, 216])


torch.Size([10, 6, 9, 24])

In [42]:
     

tc = TestConv()
input = torch.randn([5,3,20])
print(tc(input).size())
label = torch.LongTensor([0, 1,0,0,0])
loss = F.cross_entropy(tc(input), label)
loss.backward()
# o = [torch.randn([10,20,30]), torch.randn(10,20,30), torch.randn(10,20,30)]

# cvl = nn.Conv2d(10,20,kernel_size=3)
# cvl.state_dict()['weight'].size()
# torch.unsqueeze(torch.randn(10),dim=0).shape

torch.Size([5, 2])


In [35]:
clusterx = torch.randn([5,20])
clustery = torch.LongTensor([1,0,1,0,1])
cml = ClusterML(2, 20, 20)

# print(o,pd)
cml.train()
optimizer = torch.optim.Adam(cml.parameters(), lr=1e-4, weight_decay=1e-3)
o, pd = cml(clusterx, clustery)
loss = F.cross_entropy(o, torch.squeeze(clustery))
loss.backward()
optimizer.step()

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: leaf variable has been moved into the graph interior

In [44]:

inputx = torch.randn([10,3,30])
# spb = Spatial_Block(2,3,4)
# print(spb(inputx).size())
# avgp1d = nn.AvgPool1d(3,1,padding=1)
# print(avgp1d(inputx).size())
lstmsq = SMATE_Encoder(input_dimension=3, input_length=30, d_prime=20, out_channels=[128,128,128], kernels=[3,3,3], final_pool=3, num_layers=3, rnn="LSTM")
print(lstmsq(inputx).size())
ln = nn.Linear(30,25)
print(ln(inputx).shape)
a1 = torch.randn(10,20)
a2 = torch.randn(20,30)
c = torch.randn(30)
(a1 @ a2+c).size()
x1 = torch.Tensor([[0,0],[0,0]])
x2 = torch.Tensor([1,2])
x1+x2

torch.Size([10, 30, 300])
torch.Size([10, 3, 25])


tensor([[1., 2.],
        [1., 2.]])

In [4]:
from models import ResNet
str(type(ResNet()).__name__)

'ResNet'

In [46]:
labels = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]
labels = np.array(labels)
uni = np.unique(labels)
dict = {}
for i in uni:
        dict[i] = [j for j,x in enumerate(labels) if x==i]
print(dict)
for i in dict:
        print(len(dict[i]))
x = torch.Tensor([[[1]],[[2]]])
y = torch.randn(2,3,2)
z = x.expand_as(y)
z

{0: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], 1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49], 3: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}
20
20
20
20


tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[2., 2.],
         [2., 2.],
         [2., 2.]]])

In [15]:
labeltest = torch.LongTensor([1,1,2,2])
labeltest[labeltest==1]
x = torch.randn([3,3])
torch.unique(labeltest)
for i in torch.unique(labeltest):
    print(x[i])
    
class LinearT(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.W = nn.Parameter(torch.randn(in_channels, out_channels), requires_grad=True)
        self.b = nn.Parameter(torch.randn(out_channels), requires_grad=True)
    
    def forward(self, x):
        N = x.size(0)
        return (x@self.W)+self.b

linear_model = LinearT(2,5)
optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.01)
for i in range(10):
    linear_model.train()
    x = torch.randn(10,2)
    y = torch.LongTensor([0 for i in range(10)])
    loss = F.cross_entropy(linear_model(x), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  



tensor([1.0039, 0.5983, 0.8335])
tensor([ 0.3076,  1.9460, -0.5611])


In [16]:
from models import SK_MultiScaleConv1dBlock
x = torch.randn(10,32,20)
model = SK_MultiScaleConv1dBlock(32,0,3)
gap = nn.AdaptiveAvgPool1d(1)
gap(model(x)).size()

torch.Size([10, 32, 1])

In [16]:
import argparse
import time
from turtle import forward

from sklearn.metrics import accuracy_score, confusion_matrix
from models import * 
from utils.utils import * 
from utils.data_loader import mitdb_aryth_batchiter
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import sys
import os 
from tqdm import tqdm

parser = argparse.ArgumentParser()

seed = 1000
random_seed(seed)
dataset_divide = [0.6, 0.2, 0.2]

class SKnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.skseq = nn.Sequential(
            nn.Conv1d(2, 32, 3, padding=1),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.AdaptiveAvgPool1d(1)
        )
        self.linear = nn.Linear(32, 5)
    def forward(self, x):
        x = torch.transpose(x, 1, 2)
        y = self.skseq(x)
        y = y.squeeze(2)
        return self.linear(y)


DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119,
122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230]
DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202,
210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]
Total_DS = DS1 + DS2
random.shuffle(Total_DS)
DS_train = Total_DS[0:int(dataset_divide[0] * len(Total_DS))]
DS_eval = Total_DS[int(dataset_divide[0] * len(Total_DS)):int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS))]
DS_test = Total_DS[int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS)):]




path  = "../data/mit-bih-arrhythmia-database-1.0.0"
model = ResNet()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)



def train(epochs):
    loss_list=[sys.maxsize]
    ttime = time.time()
    for epoch in tqdm(range(epochs)):
        train_batch_iter = mitdb_aryth_batchiter(10, DS_train, path, use_last_batch=False)
        epoch_time = time.time()
        

        model.train()
        loss_total = []
        for inputs, labels in train_batch_iter:
            t = time.time()
            optimizer.zero_grad()
            output = model(inputs)
            loss = F.cross_entropy(output, labels)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
            loss_total.append(loss.item())

        model.eval()
        eval_output = []
        eval_label = []
        eval_batch_iter = mitdb_aryth_batchiter(10, DS_eval, path)
        for inputs, labels in eval_batch_iter:
            output = model(inputs)
            eval_output.append(output)
            eval_label.append(labels)

        # eval_pred = torch.cat(eval_output, dim=0)
        # eval_pred = eval_pred.max(1)[1].cpu().numpy()

        acc_val = accuracy(torch.cat(eval_output,dim=0), torch.cat(eval_label, dim=0))
        if epoch%2==0:
            print('Epoch: {:04d}'.format(epoch + 1),
            'acc_val:{:.4f}'.format(acc_val),
            'time: {:.4f}s'.format(time.time() - epoch_time),
            'total_time: {:.4f}min'.format((time.time()-ttime)/60)
             )




train(50)






  2%|▏         | 1/50 [01:23<1:08:20, 83.69s/it]

Epoch: 0001 acc_val:0.8952 time: 4.0783s total_time: 1.3948min


  6%|▌         | 3/50 [03:01<43:38, 55.72s/it]  

Epoch: 0003 acc_val:0.8952 time: 3.5035s total_time: 3.0313min


 10%|█         | 5/50 [04:15<33:33, 44.73s/it]

...

 94%|█████████▍| 47/50 [25:11<01:25, 28.55s/it]

Epoch: 0047 acc_val:0.8952 time: 3.3967s total_time: 25.1840min


 98%|█████████▊| 49/50 [26:08<00:28, 28.53s/it]

Epoch: 0049 acc_val:0.8952 time: 3.4235s total_time: 26.1360min


100%|██████████| 50/50 [26:36<00:00, 31.94s/it]


In [8]:
save_dir = "./"+str(type(model).__name__)+ str(50)
mkdir(save_dir)
torch.save(model, save_dir+'/model.pth')

In [6]:
def test():
    test_output = []
    test_label = []
    test_batch_iter = mitdb_aryth_batchiter(10, DS_test, path)
    for inputs, labels in test_batch_iter:
            output = model(inputs)
            test_output.append(output)
            test_label.append(labels)
    acc_val = accuracy(torch.cat(test_output,dim=0), torch.cat(test_label, dim=0))
    labels = torch.cat(test_label,dim=0).cpu().numpy()
    pred = torch.cat(test_output,dim=0).max(1)[1].cpu().numpy()
    print(acc_val)
    print(confusion_matrix(labels, pred))
test()

0.9995113608600049
[[12273     0     0]
 [    1     0     0]
 [    5     0     0]]


In [59]:
# 改变字体
str = "REVERSAL NO LONGER MATTERS: ATTENTION-BASED ARRHYTHMIA DETECTION WITH LEAD-REVERSAL ECG DATA"
str.lower()

'reversal no longer matters: attention-based arrhythmia detection with lead-reversal ecg data'

In [13]:
def second2time(second):
    intsecond = int(second)
    day = int(second) // (24 * 60 * 60)
    intsecond -= day * (24 * 60 * 60)
    hour = intsecond // (60 * 60)
    intsecond -= hour * (60 * 60)
    minute = intsecond // 60
    intsecond -= 60 * minute
    return (day, hour, minute, second - int(second) + intsecond)
        
day,hour,minute,second = second2time(59.4)

print("time: {:02d}d{:02d}h{:02d}m{:.1f}s".format(day, hour, minute, second))

time: 00d00h00m59.4s


In [14]:
b1 = torch.randn(10,3,4)
b2 = torch.randn(10,4,5)
c = torch.bmm(b1, b2)
c.size()
zero_mask = torch.triu(torch.ones(5,5), diagonal=0)
zero_mask

tensor([[1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.]])

In [14]:
print("{:2.3f}".format(4.6))

4.600


In [16]:
print(3)

In [22]:
from models import * 
class SKnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.skseq = nn.Sequential(
            TemporalAttention(258, 2, 8, 8),
            nn.Conv1d(2, 32, 3, padding=1),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            TemporalAttention(258, 32, 32, 32),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.AdaptiveAvgPool1d(1)
        )
        self.linear = nn.Linear(32, 5)
    def forward(self, x):
        x = torch.transpose(x, 1, 2)
        y = self.skseq(x)
        y = y.squeeze(2)
        return self.linear(y)

model = SKnet()
save_dir = "./{}{}".format(str(type(model).__name__), str(50))
save_dir

In [26]:
torch.load(model, save_dir + "/model.pth")

ModuleAttributeError: 'SKnet' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [27]:
import argparse
import time
from turtle import forward

from sklearn.metrics import accuracy_score, confusion_matrix
from models import * 
from utils.utils import * 
from utils.data_loader import mitdb_aryth_batchiter
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import sys
import os 

sys.stdout = Logger('log.log')

parser = argparse.ArgumentParser()

seed = 503
random_seed(seed)
dataset_divide = [0.6, 0.2, 0.2]

class SKnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.skseq = nn.Sequential(
            TemporalAttention(258, 2, 8, 8),
            nn.Conv1d(2, 32, 3, padding=1),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            TemporalAttention(258, 32, 32, 32),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.ReLU(),
            SK_MultiScaleConv1dBlock(32, 0.8),
            nn.AdaptiveAvgPool1d(1)
        )
        self.linear = nn.Linear(32, 5)
    def forward(self, x):
        x = torch.transpose(x, 1, 2)
        y = self.skseq(x)
        y = y.squeeze(2)
        return self.linear(y)


DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119,
122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 223, 230]
DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202,
210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]
Total_DS = DS1 + DS2
random.shuffle(Total_DS)
DS_train = Total_DS[0:int(dataset_divide[0] * len(Total_DS))]
DS_eval = Total_DS[int(dataset_divide[0] * len(Total_DS)):int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS))]
DS_test = Total_DS[int((dataset_divide[1] + dataset_divide[0]) * len(Total_DS)):]




path  = "../data/mit-bih-arrhythmia-database-1.0.0"
model = ResNet()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)


model = model.cuda()
def train(epochs, use_gpu=True):
    loss_list=[sys.maxsize]
    ttime = time.time()
    for epoch in range(epochs):
        epoch_time = time.time()
        train_batch_iter = mitdb_aryth_batchiter(10, DS_train, path, use_last_batch=False)
        
        

        model.train()
        loss_total = []
        for inputs, labels in train_batch_iter:
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            output = model(inputs)
            loss = F.cross_entropy(output, labels)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
            loss_total.append(loss.item())

        model.eval()
        eval_output = []
        eval_label = []
        eval_batch_iter = mitdb_aryth_batchiter(10, DS_eval, path)
        for inputs, labels in eval_batch_iter:
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            output = model(inputs)
            eval_output.append(output)
            eval_label.append(labels)

        # eval_pred = torch.cat(eval_output, dim=0)
        # eval_pred = eval_pred.max(1)[1].cpu().numpy()

        acc_val = accuracy(torch.cat(eval_output,dim=0), torch.cat(eval_label, dim=0))
        total_time = time.time() - ttime
        day, hour, minute, second = second2time(total_time)
        if epoch%2==0:
            print('Epoch: {:04d}'.format(epoch + 1),
            'acc_val:{:.4f}'.format(acc_val),
            'epoch_time: {:.2f}s'.format(time.time() - epoch_time),
            'total_time: {:02d}d{:02d}h{:02d}m{:.1f}s'.format(day, hour, minute, second)
             )




train(50)
save_dir = "./{}{}".format(str(type(model).__name__), str(50))
mkdir(save_dir)
torch.save(model, save_dir + '/' + 'model.pth')


def test(use_gpu=True):
    model.eval()
    test_output = []
    test_label = []
    test_batch_iter = mitdb_aryth_batchiter(10, DS_test, path)
    for inputs, labels in test_batch_iter:
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        output = model(inputs)
        test_output.append(output)
        test_label.append(labels)
    acc_val = accuracy(torch.cat(test_output,dim=0), torch.cat(test_label, dim=0))
    labels = torch.cat(test_label,dim=0).cpu().numpy()
    pred = torch.cat(test_output,dim=0).max(1)[1].cpu().numpy()
    print(acc_val)
    print(confusion_matrix(labels, pred))
test()


AttributeError: 'str' object has no attribute 'formt'

In [5]:
#-----------------------------------------------------------#

#   该函数用于triplet loss,将

#-----------------------------------------------------------#
def triplet_loss(embeddings, nclass):
    """Choose the hard positive and negative samples to generate the triplet 选取难正样本和负样本,用于生成triplet_loss

    This function is delighted by the paper 'FaceNet: A Unified Embedding for Face Recognition and Clustering' to find the proper triplets in a batch. Hence, the batch should contain the different class samples. The input is the number of class mul the number of samples contained in each class. 这个函数受启发于论文'FaceNet: A Unified Embedding for Face Recognition and Clustering',用于寻找一个batch中的不同triplets.因此输入的每一个batch应该包含每一个种类的样本,输入大小等同于种类数*每个种类包含的样本数.

    Args:
      embeddings:
        the embeddings, which size is batch_size x samples   
      
    Returns:


    """
    batch_size = embeddings[0]
    x = embeddings.unsqueeze(0).expand(embeddings.size(0), embeddings.size(0), embeddings.size(1))
    y = embeddings.unsqueeze(1).expand(embeddings.size(0), embeddings.size(0), embeddings.size(1))

    distance_metric = torch.pow(x - y, 2).sum(2)
    return distance_metric
    

In [3]:


a = torch.LongTensor([[1,0,0,0],[0,1,0,1],[1,0,1,0]]) 
a.size()
b = a.unsqueeze(1).expand(a.size(0), a.size(0), a.size(1))
c = a.unsqueeze(0).expand(a.size(0), a.size(0), a.size(1))
torch.pow(b - c, 2).sum(2)

tensor([[0, 3, 1],
        [3, 0, 4],
        [1, 4, 0]])

In [11]:
labels = np.array([0,2,1,0,1,0])
c = [j for j,x in enumerate(labels) if x==1]
np.max(np.bincount(labels))

3

In [2]:
import sys
a = [12][sys.maxsize//12]

IndexError: list index out of range